# CmdStanPy Demo: One-Patient Stand ODE Model

This notebook builds a toy one-patient dataset, compiles the Stan ODE model from `stand_ode.stan`, and runs a short CmdStanPy sampling job.


The workflow below is intentionally lightweight so the model stays fast to iterate on:

- ensure CmdStan is available for CmdStanPy
- create a synthetic longitudinal patient record
- feed the data to the Stan model in `content/ipynb/stand_ode.stan`
- compile the model, draw a few MCMC chains, and inspect key diagnostics

Running the notebook end-to-end yields posterior samples and posterior predictive summaries for the example patient.


In [7]:
from pathlib import Path

import numpy as np
import pandas as pd

import cmdstanpy


In [9]:
cmdstanpy.install_cmdstan()

CmdStan install directory: /home/dulunche/.cmdstan
Installing CmdStan version: 2.37.0
Download successful, file: /tmp/tmplqkxu3ed
Extracting distribution
Unpacked download as cmdstan-2.37.0
Building version cmdstan-2.37.0, may take several minutes, depending on your system.
Installed cmdstan-2.37.0
Test model compilation


True

In [10]:
from cmdstanpy import CmdStanModel, cmdstan_path

try:
    CMDSTAN_PATH = cmdstan_path()
except ValueError:
    from cmdstanpy.install_cmdstan import install_cmdstan

    CMDSTAN_PATH = install_cmdstan()

print(f"Using CmdStan installation at: {CMDSTAN_PATH}")

Using CmdStan installation at: /home/dulunche/.cmdstan/cmdstan-2.37.0


In [11]:
patient_df = pd.DataFrame(
    {
        "time": [0.0, 7.0, 14.0, 28.0, 56.0, 84.0],
        "bcva": [45.0, 47.5, 50.0, 54.0, 57.0, 58.5],
    }
)

patient_df


,time,bcva
0,0.0,45.0
1,7.0,47.5
2,14.0,50.0
3,28.0,54.0
4,56.0,57.0
5,84.0,58.5


In [12]:
stan_data = {
    "N": len(patient_df),
    "time": patient_df["time"].to_list(),
    "bcva_obs": patient_df["bcva"].to_list(),
    "start_t": float(patient_df["time"].iloc[0]),
    "lconc0": 1.6,
    "K": 0.0045,
    "hill": 4.0,
    "r180": 0.65,
    "beta": 0.35,
    "sigma_prior_scale": 5.0,
}

stan_data


{'N': 6,
 'time': [0.0, 7.0, 14.0, 28.0, 56.0, 84.0],
 'bcva_obs': [45.0, 47.5, 50.0, 54.0, 57.0, 58.5],
 'start_t': 0.0,
 'lconc0': 1.6,
 'K': 0.0045,
 'hill': 4.0,
 'r180': 0.65,
 'beta': 0.35,
 'sigma_prior_scale': 5.0}

In [15]:
import tempfile
import shutil

def find_repo_root(marker: str = "config.toml") -> Path:
    # Walk up the directory tree until the project root is found.
    here = Path.cwd()
    for candidate in [here, *here.parents]:
        if (candidate / marker).exists():
            return candidate
    raise FileNotFoundError(f"Could not locate `{marker}` starting from {here}.")

repo_root = find_repo_root()
stan_file_orig = repo_root / "content" / "ipynb" / "stand_ode.stan"

# Copy Stan file to /tmp to avoid path issues with CmdStan's makefile
tmp_dir = Path("/tmp/stan_models")
tmp_dir.mkdir(exist_ok=True)
stan_file = tmp_dir / "stand_ode.stan"
shutil.copy(stan_file_orig, stan_file)

print(f"Compiling model at: {stan_file}")
stand_ode_model = CmdStanModel(stan_file=str(stan_file))


10:14:34 - cmdstanpy - INFO - compiling stan file /tmp/stan_models/stand_ode.stan to exe file /tmp/stan_models/stand_ode


Compiling model at: /tmp/stan_models/stand_ode.stan


ValueError: Failed to compile Stan model '/tmp/stan_models/stand_ode.stan'. Console:

--- Translating Stan model to C++ code ---
bin/stanc --filename-in-msg=stand_ode.stan --o=/tmp/stan_models/stand_ode.hpp /tmp/stan_models/stand_ode.stan
Syntax error in 'stand_ode.stan', line 2, column 9 to column 10, parsing error:
   -------------------------------------------------
     1:  functions {
     2:    vector[1] drug_disease_stim_kinL_Et_ode(real t,
                  ^
     3:                                            vector[1] y,
     4:                                            array[] real theta,
   -------------------------------------------------

"[" (list of commas) "]" expected in unsized return type of function definition.
make: *** [make/program:66: /tmp/stan_models/stand_ode.hpp] Error 1

Command ['make', 'STANCFLAGS+=--filename-in-msg=stand_ode.stan', '/tmp/stan_models/stand_ode']
	exited with code '2' No such file or directory


In [ ]:
fit = stand_ode_model.sample(
    data=stan_data,
    seed=24531,
    chains=4,
    parallel_chains=4,
    iter_warmup=500,
    iter_sampling=500,
    show_progress=True,
)

fit


In [ ]:
summary = fit.summary()
summary.loc[["k_in", "k_out", "emax0", "lec50", "sigma", "R0"]]


In [ ]:
posterior_bcva = fit.stan_variable("bcva_rep")
mean_bcva = posterior_bcva.mean(axis=0)
lower, upper = np.percentile(posterior_bcva, [5, 95], axis=0)

import matplotlib.pyplot as plt

plt.figure(figsize=(8, 4))
plt.plot(patient_df["time"], patient_df["bcva"], "o", label="Observed", color="tab:blue")
plt.plot(patient_df["time"], mean_bcva, label="Posterior mean", color="tab:orange")
plt.fill_between(
    patient_df["time"],
    lower,
    upper,
    color="tab:orange",
    alpha=0.2,
    label="90% posterior interval",
)
plt.xlabel("Time (days)")
plt.ylabel("BCVA score")
plt.title("Posterior predictive check")
plt.legend()
plt.tight_layout()
plt.show()
